# Using SQL queries on DataFrames

- - -

#### Declarative Manipulation (SQL)
* Advantage: You need to describe only **what** is the result you want.
* Disadvantage: SQL does not have primitives for common analysis operations such as **covariance**


- - -

* Spark supports a subset of the Hive SQL query language.
* For example, You can use Hive select syntax to select a subset of the rows in a dataframe.
* To use sql on a dataframe you need to ``first register it as a TempTable``
* For variety, we are using here a small dataframe loaded from a JSON file.

### pyspark import & SparkContext 생성

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType

sc = SparkContext(master="local[*]")
print(sc)

# Just like using Spark requires having a SparkContext, using SQL requires an SQLContext
sqlContext = SQLContext(sc)
print(sqlContext)

<SparkContext master=local[*] appName=pyspark-shell>


In [2]:
import urllib.request
# people.json(예제파일 다운로드)
f = urllib.request.urlretrieve ("https://docs.google.com/uc?export=download&id=1TZyM7Gfc6XWLot-L36TDV-JwySgHxGv4", "people.json")
data_file = "./people.json"

In [3]:
# Create a DataFrame from the file(s) pointed to by path
people = sqlContext.read.json(data_file)

# The inferred schema can be visualized using the printSchema() method~
people.show()
people.printSchema()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



## ★Register the DataFrame as a table★

* **Registers a DataFrame as a Temporary Table in the SQLContext**
* Usage : ``registerTempTable(tableName)``
* Arguments :
```
talbeName : A character vector containing the name of the table
```
* Example : [DataFrame_Spark].registerTempTable("hi_hi_hi")


In [4]:
## Register this DataFrame as a table
people.registerTempTable("people")

## SQL statements can be run by using the sql methods provided by sqlContext
teenagers = sqlContext.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")

# show를 사용했을 때, DataFrame return
print("show를 사용하면....")
teenagers.show()

# collect를 사용했을 때, Row형태의 RDD return
print("\ncollect를 사용하면....")
print(teenagers.collect())

show를 사용하면....
+------+
|  name|
+------+
|Justin|
+------+


collect를 사용하면....
[Row(name='Justin')]


### Excercise 1 - lec6 복습!!(20 point)

- - -
**task**

* **출력**과 같은 값이 나오도록 code를 작성하세요 (개당 4 point)
- - -
**출력**

```
1 ====>  [Row(name='Justin')]
2 ====>  Justin
3 ====>  Row(name='Justin')
4 ====> 
+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

5 ====> 
+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
+---+------+
```

In [8]:
from pyspark.sql.functions import col


print('1 ====> ', teenagers.collect())#첫 열을 출력
print('2 ====> ',teenagers.collect()[0]['name'])#첫 열의 name부분을 출력
print('3 ====> ', teenagers.collect()[0])#첫 열의 원소 0번쨰를 출력
print("4 ====> ") #filter함수를 통해 age라는 행에서 30인 사람만 출력
people.filter(col("age") == 30).show()
print("5 ====> ") #filter함수를 통해 age라는 행에서 1이상인 사람만 출력
people.filter(col("age") > 1).show()

1 ====>  [Row(name='Justin')]
2 ====>  Justin
3 ====>  Row(name='Justin')
4 ====> 
+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

5 ====> 
+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
+---+------+



## SQL Basic

---

SQL에 대해 기본적인 사용법을 학습합니다.

* SQL 구문
```
SELECT [열] 
FROM [테이블] 
WHERE [조건]
```

In [9]:
!pip install googledrivedownloader

In [10]:
# NY weather download & register
from os.path import exists
from google_drive_downloader import GoogleDriveDownloader as gdd
import tarfile

if exists("./NY.tgz"):
    !rm -rf ./NY.tgz
if exists("./NY.parquet"):
    !rm -rf ./NY.parquet
    
gdd.download_file_from_google_drive(file_id='1hAHV6vC6FvVgrYnoN-lR-IfH488-H121',
                                   dest_path = './NY.tgz')
!tar -xzvf NY.tgz

# read & register
df = sqlContext.read.load("NY.parquet")
df.registerTempTable("weather")

NY.parquet/
NY.parquet/_SUCCESS
NY.parquet/part-00022-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00000-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00021-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00001-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00023-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00002-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00024-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00003-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00025-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00004-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00027-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00005-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-0000

### (1) DESC
* 테이블 구조 확인
* ``[DataFrame].printSchema``와 동일한 기능

In [11]:
# pyspark code
df.printSchema()

# SQL code
query= """DESC weather"""#큰 따옴표 무조건 3개씩 작성할 것
sqlContext.sql(query).show()

root
 |-- Station: string (nullable = true)
 |-- Measurement: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Values: binary (nullable = true)
 |-- dist_coast: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|    Station|   string|   null|
|Measurement|   string|   null|
|       Year|   bigint|   null|
|     Values|   binary|   null|
| dist_coast|   double|   null|
|   latitude|   double|   null|
|  longitude|   double|   null|
|  elevation|   double|   null|
|      state|   string|   null|
|       name|   string|   null|
+-----------+---------+-------+



### (2) FIRST와 LAST
* ``FIRST`` : 첫 번째 ROW
* ``LAST`` : 마지막 ROW

In [13]:
# pyspark code
from pyspark.sql.functions import col, first, last
df.select(first(col("Station")), last(col("Station"))).show()

# SQL code
print("★★★★★In SQL★★★★★★")
query = """SELECT FIRST(Station), LAST(Station) FROM weather"""
sqlContext.sql(query).show(3)

+---------------------+--------------------+
|first(Station, false)|last(Station, false)|
+---------------------+--------------------+
|          USW00094704|         USC00307664|
+---------------------+--------------------+

★★★★★In SQL★★★★★★
+---------------------+--------------------+
|first(Station, false)|last(Station, false)|
+---------------------+--------------------+
|          USW00094704|         USC00307664|
+---------------------+--------------------+



### (3) SELECT와 AS
* ``SELECT`` : 특정 column 선택
* ``AS``     : 특정 column의 이름 변경
* 사용팁 : ``select``와 동시에 ``as``를 이용하여 column의 이름을 변경한다!

In [14]:
# pyspark code
df.select(col("Station").alias("JINOO_Station"), col("name").alias("name_JW")).show(3)

# SQL code
print("★★★★★In SQL★★★★★★")
query = """SELECT Station AS JINOO_Station, name AS name_JW FROM weather"""
sqlContext.sql(query).show(3)

+-------------+-----------------+
|JINOO_Station|          name_JW|
+-------------+-----------------+
|  USW00094704|DANSVILLE MUNI AP|
|  USW00094704|DANSVILLE MUNI AP|
|  USW00094704|DANSVILLE MUNI AP|
+-------------+-----------------+
only showing top 3 rows

★★★★★In SQL★★★★★★
+-------------+-----------------+
|JINOO_Station|          name_JW|
+-------------+-----------------+
|  USW00094704|DANSVILLE MUNI AP|
|  USW00094704|DANSVILLE MUNI AP|
|  USW00094704|DANSVILLE MUNI AP|
+-------------+-----------------+
only showing top 3 rows



### (4) SELECT와 DISTINCT
* ``SELECT`` : 특정 column 선택
* ``DISTICNT``     : 결과값에 중복된 데이터가 있으면 중복되는 N개의 데이터는 1건으로 처리해서 출력

In [15]:
# pyspark code
df.select(col("Station").alias("JINOO_Station_DISTINCT"), col("name").alias("name_JW")).distinct().show(3)

# SQL code
print("★★★★★In SQL★★★★★★")
query = """SELECT DISTINCT Station AS JINOO_Station_DISTINCT, name AS name_JW FROM weather"""
sqlContext.sql(query).show(3)

+----------------------+--------------------+
|JINOO_Station_DISTINCT|             name_JW|
+----------------------+--------------------+
|           USC00300505|       BEAVER MEADOW|
|           USC00305426|         MOHONK LAKE|
|           USC00305798|NEW YORK BENSONHURST|
+----------------------+--------------------+
only showing top 3 rows

★★★★★In SQL★★★★★★
+----------------------+--------------------+
|JINOO_Station_DISTINCT|             name_JW|
+----------------------+--------------------+
|           USC00300505|       BEAVER MEADOW|
|           USC00305426|         MOHONK LAKE|
|           USC00305798|NEW YORK BENSONHURST|
+----------------------+--------------------+
only showing top 3 rows



### (5) SELECT와 COUNT, AVG, SUM, MAX, MIN
* 이름 그대로의 역할!!
* SELECT와 연계하여 쓴다.

In [16]:
# pyspark code
from pyspark.sql.functions import count, avg, sum, max, min
df.select(count(col("Station"))).show()
df.select(avg(col("dist_coast")), sum(col("dist_coast"))).show()
df.select(min(col("elevation")), max(col("elevation"))).show()


# SQL code
print("★★★★★In SQL★★★★★★")
query1 = """SELECT COUNT(Station) FROM weather"""
query2 = """SELECT AVG(dist_coast), SUM(dist_coast) FROM weather"""
query3 = """SELECT MIN(elevation), MAX(elevation) FROM weather"""

sqlContext.sql(query1).show(3)
sqlContext.sql(query2).show(3)
sqlContext.sql(query3).show(3)

+--------------+
|count(Station)|
+--------------+
|        168398|
+--------------+

+------------------+-------------------+
|   avg(dist_coast)|    sum(dist_coast)|
+------------------+-------------------+
|245.78455113006692|4.138962684120101E7|
+------------------+-------------------+

+------------------+-----------------+
|    min(elevation)|   max(elevation)|
+------------------+-----------------+
|-999.9000244140625|838.2000122070312|
+------------------+-----------------+

★★★★★In SQL★★★★★★
+--------------+
|count(Station)|
+--------------+
|        168398|
+--------------+

+------------------+-------------------+
|   avg(dist_coast)|    sum(dist_coast)|
+------------------+-------------------+
|245.78455113006692|4.138962684120101E7|
+------------------+-------------------+

+------------------+-----------------+
|    min(elevation)|   max(elevation)|
+------------------+-----------------+
|-999.9000244140625|838.2000122070312|
+------------------+-----------------+



### (6) WHERE
* **조건절** 
* pyspark의 ``filter``와 동일한 기능
* example

```
# Example 1: weather 테이블에서 elevationrk가 800이상인 모든 column을 가져오되 중복을 제거해라
SELECT DISTINCT * FROM weather WHERE elevation >= 800

# Example 2: weather 테이블에서 elevation가 100 미만인 Station column을 가져와라
SELECT Station FROM weather WHERE elevation < 100

# Example 3: weather 테이블에서 elevation가 600이상이고 800 미만인 모든 column을 가져오되 중복을 제거해라!!"
SELECT DISTINCT * FROM weather WHERE elevation >= 600 AND elevation < 800

```

* example 외에도 사용법이 너무 많습니다. [눌러1](https://loveiskey.tistory.com/58), [눌러2](http://www.sqlprogram.com/Basics/sql-where.aspx), [눌러3](https://www.guru99.com/where-clause.html)을 참고해주세요.!

In [17]:
# SQL code
print("Example 1 : weather 테이블에서 elevationrk가 800이상인 모든 column을 가져오되 중복을 제거해라")
query1 = """SELECT DISTINCT * FROM weather WHERE elevation >= 800"""
sqlContext.sql(query1).show(2)

print("Example 2: weather 테이블에서 elevation가 100 미만인 Station column을 가져와라")
query2 = """SELECT Station FROM weather WHERE elevation < 100"""
sqlContext.sql(query2).show(2)

print("Example 3: weather 테이블에서 elevation가 600이상이고 800 미만인 모든 column을 가져오되 중복을 제거해라!!")
query3 = """SELECT DISTINCT * FROM weather WHERE elevation >= 600 AND elevation < 800"""
sqlContext.sql(query3).show(2)

Example 1 : weather 테이블에서 elevationrk가 800이상인 모든 column을 가져오되 중복을 제거해라
+-----------+-----------+----+--------------------+------------------+------------------+------------------+-----------------+-----+-----------+
|    Station|Measurement|Year|              Values|        dist_coast|          latitude|         longitude|        elevation|state|       name|
+-----------+-----------+----+--------------------+------------------+------------------+------------------+-----------------+-----+-----------+
|USC00307799|   TMAX_s20|2005|[02 C9 8F C9 0C C...| 128.7790069580078|42.016700744628906|-74.41670227050781|807.7000122070312|   NY|  SLIDE MTN|
|USC00304520|       SNOW|1922|[00 00 00 00 00 0...|285.82501220703125|44.099998474121094|             -74.0|838.2000122070312|   NY|LAKE COLDEN|
+-----------+-----------+----+--------------------+------------------+------------------+------------------+-----------------+-----+-----------+
only showing top 2 rows

Example 2: weather 테이블에서 elevation

### (7) GROUP BY와 HAVING
* GROUP BY문은 동일한 값을 가진 데이터를 집계해서 조회하고자 할 때 사용하는 문장이다.
* 작성 방법

```
SELECT  [구문]
FROM    [구문]
GROUP BY [구문]
HAVING  [구문]

- 집계할 컬럼을 GROUP BY절 뒤에 적어준다.
- SELECT절에는 GROUP BY에 명시된 컬럼만 사용할 수 있다.
- HAVING에는 GROUP BY에 조건을 추가한다
```

In [18]:
# pyspark code
df.groupby(col("Measurement")).agg({"Measurement" : "count", "year": "min"})\
  .withColumnRenamed("count(Measurement)", "count")\
  .withColumnRenamed("min(year)", "MinYear")\
  .show(5)

# SQL code
print("★★★★★In SQL★★★★★★")
query = """SELECT Measurement, COUNT(Measurement) AS count, MIN(year) AS MinYear
           FROM weather
           GROUP BY Measurement"""
sqlContext.sql(query).show(5)

+-----------+-----+-------+
|Measurement|count|MinYear|
+-----------+-----+-------+
|   TMIN_s20|13442|   1873|
|       TMIN|13442|   1873|
|   SNOW_s20|15629|   1884|
|       TOBS|10956|   1876|
|   SNWD_s20|14617|   1888|
+-----------+-----+-------+
only showing top 5 rows

★★★★★In SQL★★★★★★
+-----------+-----+-------+
|Measurement|count|MinYear|
+-----------+-----+-------+
|   TMIN_s20|13442|   1873|
|       TMIN|13442|   1873|
|   SNOW_s20|15629|   1884|
|       TOBS|10956|   1876|
|   SNWD_s20|14617|   1888|
+-----------+-----+-------+
only showing top 5 rows



In [65]:
# year를 GROUP BY를 통해 그룹화 하고, year 별 dist_coast와 elevation의 평균을 구하고
# year를 기준으로 오름차순 한다.
from pyspark.sql.functions import desc
df.groupby(col("year")).agg({"dist_coast":"avg","elevation":"avg"})\
  .sort(desc("year")).show(5)


# SQL code
print("★★★★★In SQL★★★★★★")
query = """SELECT year, AVG(elevation), AVG(dist_coast)
           FROM weather
           GROUP BY year
           ORDER BY year DESC
           """
sqlContext.sql(query).show(5)

+----+------------------+------------------+
|year|    avg(elevation)|   avg(dist_coast)|
+----+------------------+------------------+
|2013| 279.4017901454714|262.15733492754947|
|2012|266.33072170511406| 257.8840703884677|
|2011| 265.4277518365087|253.01556445820043|
|2010|262.55130806776833|248.06859466340276|
|2009| 261.5488291051326|246.52965264467915|
+----+------------------+------------------+
only showing top 5 rows

★★★★★In SQL★★★★★★
+----+------------------+------------------+
|year|    avg(elevation)|   avg(dist_coast)|
+----+------------------+------------------+
|2013| 279.4017901454714|262.15733492754947|
|2012|266.33072170511406| 257.8840703884677|
|2011| 265.4277518365087|253.01556445820043|
|2010|262.55130806776833|248.06859466340276|
|2009| 261.5488291051326|246.52965264467915|
+----+------------------+------------------+
only showing top 5 rows



In [66]:
# SQL code
# HAVING 추가
# year를 GROUP BY를 통해 그룹화 하고, year 별 dist_coast와 elevation의 평균을 구하고
# dist_coast의 평균이 256 이상이고 elevation의 평균이 260 이상인 ROW를 구하되
# year를 기준으로 오름차순 한다.
print("★★★★★In SQL★★★★★★")
query = """SELECT year, AVG(elevation), AVG(dist_coast)
           FROM weather
           GROUP BY year
           HAVING AVG(dist_coast) >= 256 AND AVG(elevation) >= 260
           ORDER BY year DESC
           """
sqlContext.sql(query).show(5)

★★★★★In SQL★★★★★★
+----+------------------+------------------+
|year|    avg(elevation)|   avg(dist_coast)|
+----+------------------+------------------+
|2013| 279.4017901454714|262.15733492754947|
|2012|266.33072170511406| 257.8840703884677|
|1915| 264.1221125600379|258.39760847437293|
|1914|262.33999927043914| 259.3964216718078|
|1913|271.95626452024027| 257.1068253879962|
+----+------------------+------------------+
only showing top 5 rows



### ★★★★WHERE와 HAVING???★★★★

query에 `WHERE`와 `HAVING`이 있다면, `WHERE`가 적용된 후*(`조건이 적용된 table`에)* `HAVING`이 적용된다.

* **반드시 다음 주소에 접속하여 차이점을 확인할 것 (2와 3).**
[http://wiki.gurubee.net/pages/viewpage.action?pageId=26743892](http://wiki.gurubee.net/pages/viewpage.action?pageId=26743892)

### Excercise 2 - GROUP BY와 HAVING 적용해보자! (30 point)

- - -
task 별로 ``GROUP BY``와 위에서 학습한 여러 ``SQL function``을 이용하여 query 를 작성하세요(task 별 10 point)

---

**task**

* 1 : ``weather``에서 ``name``별 가장 최근 ``year``와 ``dist_coast``, ``elevation``의 평균을 구하고 가장 최근 ``name``를 기준으로 정렬(default) 하는 query를 작성합니다. 단, 가장 최근 ``year``의 이름은 ``maxYear``, 평균 ``dist_coast``와 ``elevation``의 이름은 ``avgDist_coast``, ``avgElevation``으로 변경하세요.(10 point)

<br>

* 2 : ``weather``에서 ``year``가 ``2000 이상``인 결과에 대해, ``name``별 가장 최근 ``year``와 ``dist_coast``, ``elevation``의 평균을 구하고 가장 최근 ``name``를 기준으로 정렬(default) 하는 query를 작성합니다. 단, 각 column명은 task1과 동일하게 적용합니다. (10 point)

<br>

* 3 : ``weather``에서 ``year``가 ``2000 이상``인 결과에 대해, ``name``별 가장 최근 ``year``와 ``dist_coast``, ``elevation``의 평균을 구합니다. 그리고 ``name``별 평균 ``dist_coast``가 175.44400024414062이거나 평균 ``elevation``이 304.5인 결과를 출력하는 query를 작성합니다. 단, 각 column명은 task1과 동일하게 적용합니다. (10 point)

- - -
**출력 예시**
```
# task1 output
+-------------+-------+------------------+-----------------+
|         name|maxYear|     avgDist_coast|     avgElevation|
+-------------+-------+------------------+-----------------+
|    ADAMS CTR|   1950|  376.802001953125|121.9000015258789|
|      ADDISON|   2013| 296.1679992675781|            304.5|
|ADDISON 1 NNE|   1988| 300.0060119628906|373.3999938964844|
|       ALBANY|   1970|166.00100708007812|              0.0|
|    ALBANY AP|   2013|175.44400024414062| 95.0999984741211|
+-------------+-------+------------------+-----------------+
only showing top 5 rows

# task2 output
+-----------+-------+------------------+------------------+
|       name|maxYear|     avgDist_coast|      avgElevation|
+-----------+-------+------------------+------------------+
|    ADDISON|   2013| 296.1679992675781|             304.5|
|  ALBANY AP|   2013|175.44400024414062|  95.0999984741211|
|ALBION 2 NE|   2012| 436.3030090332031|134.10000610351562|
| ALCOVE DAM|   2013|152.88999938964844|             185.0|
|     ALFRED|   2013| 330.6700134277344|             520.0|
+-----------+-------+------------------+------------------+
only showing top 5 rows

# task3 output
+---------+-------+------------------+----------------+
|     name|maxYear|     avgDist_coast|    avgElevation|
+---------+-------+------------------+----------------+
|ALBANY AP|   2013|175.44400024414062|95.0999984741211|
|  ADDISON|   2013| 296.1679992675781|           304.5|
+---------+-------+------------------+----------------+
```

**task**
* 1 : ``weather``에서 ``name``별 가장 최근 ``year``와 ``dist_coast``, ``elevation``의 평균을 구하고 가장 최근 ``name``를 기준으로 정렬(default) 하는 query를 작성합니다. 단, 가장 최근 ``year``의 이름은 ``maxYear``, 평균 ``dist_coast``와 ``elevation``의 이름은 ``avgDist_coast``, ``avgElevation``으로 변경하세요.(10 point)

In [20]:
# 2-1 답 작성
task1_query = """SELECT name, MAX(year) AS maxYear, AVG(dist_coast) AS avgDist_coast, 
                        AVG(elevation) AS avgElevation 
                 FROM weather
                 GROUP BY name
                 ORDER BY name"""
#name과 year의 가장 최근을 뽑기 위해 max함수를 써주고 이름을 maxYear로 바꿈 또한 dist_coast행의 평균을 내고 avgDist_coast로 바꾸고 
#그 자료들은 weather에서 가져오고 name별로 묶는다.

# output
sqlContext.sql(task1_query).show(5)

+-------------+-------+------------------+-----------------+
|         name|maxYear|     avgDist_coast|     avgElevation|
+-------------+-------+------------------+-----------------+
|    ADAMS CTR|   1950|  376.802001953125|121.9000015258789|
|      ADDISON|   2013| 296.1679992675781|            304.5|
|ADDISON 1 NNE|   1988| 300.0060119628906|373.3999938964844|
|       ALBANY|   1970|166.00100708007812|              0.0|
|    ALBANY AP|   2013|175.44400024414062| 95.0999984741211|
+-------------+-------+------------------+-----------------+
only showing top 5 rows



**task**
* 2 : ``weather``에서 ``year``가 ``2000 이상``인 Row에 대해, ``name``별 가장 최근 ``year``와 ``dist_coast``, ``elevation``의 평균을 구하고 가장 최근 ``name``를 기준으로 정렬(default) 하는 query를 작성합니다. 단, 각 column명은 task1과 동일하게 적용합니다. (10 point)

In [77]:
# 2-2 답 작성
task2_query = """SELECT name, MAX(year) AS maxYear, AVG(dist_coast) AS avgDist_coast, 
                        AVG(elevation) AS avgElevation 
                 FROM weather
                 WHERE year >= 2000
                 GROUP BY name
                 ORDER BY name"""
#name과 최근 해를 뽑기 위해 max함수를 써주고 maxYear로 이름을 바꿔주고 dist_coast의 평균을 구하고 avgDist_coast로 이름을 변경
#elevation의 평균을 구해주고 avgElevation으로 변경 weahter로부터 데이터를 가져오고 year가 2000이상인 조건을 가지고 이름별로 묶는다
# output
sqlContext.sql(task2_query).show(5)

+-----------+-------+------------------+------------------+
|       name|maxYear|     avgDist_coast|      avgElevation|
+-----------+-------+------------------+------------------+
|    ADDISON|   2013| 296.1679992675781|             304.5|
|  ALBANY AP|   2013|175.44400024414062|  95.0999984741211|
|ALBION 2 NE|   2012| 436.3030090332031|134.10000610351562|
| ALCOVE DAM|   2013|152.88999938964844|             185.0|
|     ALFRED|   2013| 330.6700134277344|             520.0|
+-----------+-------+------------------+------------------+
only showing top 5 rows



**task**
* 3 : ``weather``에서 ``year``가 ``2000 이상``인 Row에 대해, ``name``별 가장 최근 ``year``와 ``dist_coast``, ``elevation``의 평균을 구합니다. 그리고 ``name``별 평균 ``dist_coast``가 175.44400024414062이거나 평균 ``elevation``이 304.5인 결과를 출력하는 query를 작성합니다. 단, 각 column명은 task1과 동일하게 적용합니다. (10 point)

In [83]:
# 2-3 답 작성
task3_query = """SELECT name, MAX(year) AS maxYear, AVG(dist_coast) AS avgDist_coast, 
                        AVG(elevation) AS avgElevation 
                 FROM weather
                 WHERE year >= 2000
                 GROUP BY name
                 HAVING avgElevation == 304.5 OR avgDist_coast == 175.44400024414062"""
#이름을 뽑고,최근 해를 가져오기 위해 max함수를 쓰고 maxYear로 이름을 바꾸고 dist_coast의 평균을 구하고 avgDist_coast로 이름을 바꾼다
#elevation의 평균을 구하고 이름을 avgElevation으로 바꾸고 데이터는 weather에서 가져오고 2000년이상이 로우라는 조건을 걸고 name별로 이름을 건다
#또한 조건으로 avgElevationdl 304.5dlrjsk avgDist_coast가 175.444...일경우의 조건을 건다.
# output
sqlContext.sql(task3_query).show(5)

+---------+-------+------------------+----------------+
|     name|maxYear|     avgDist_coast|    avgElevation|
+---------+-------+------------------+----------------+
|ALBANY AP|   2013|175.44400024414062|95.0999984741211|
|  ADDISON|   2013| 296.1679992675781|           304.5|
+---------+-------+------------------+----------------+



## Exercise 3 

### DataFrame과 SQL을 사용하여 HW4 Exercise 3 다시 풀기! (25 point)

- - -
 
다음 데이터에 대하여 다음 과제를 수행하세요.

- regular.csv : KBO에서 활약한 타자들의 역대 정규시즌 성적을 포함하여 몸무게, 키 ,생년월일 등의 기본정보

**위의 데이터는 `,`로 구분되어 있습니다.**

 - **데이터의 자세한 설명은 다음의 링크를 참조해주세요.([여기를 눌러서 12. 데이터 설명 참고](https://dacon.io/cpt6/62885))**
 - 또한 regular.csv와 pre.csv를 직접 열어서 데이터가 어떻게 저장되어 있는지 확인해주세요.


- - -
**task**

- 1. sqlContext.read.csv를 이용하여 ``regular.csv``를 DataFrame으로 만든 후 . ``registerTempTable("task1_table")``을 이용하여 table로 등록합니다. 그리고 SQL의 ``DESC``를 활용하여 구조를 확인합니다(5 point)

<br>

- 2. task1에서 생성된 ``task1_table``에서, 타율(avg) column의 type을 String에서 Double로 변환 후 ``registerTempTable("task2_table")``을 사용하여 새로운 table로 등록합니다(10 point)

<br>

- 3. task2에서 생성된 ``task2_table``에서 타율(avg)이 0.300을 초과하는 모든 선수에 대해(중복포함), **각 타자의 3할을 친 횟수가 8 이상인 타자의 이름(batter_name)과 3할을 친 횟수(avgCount)를 구합니다**. 이에 덧붙여서, 3할을 친 횟수를 기준으로 내림차순을 적용하는 query를 작성하고 출력합니다.(10 point)

In [9]:
import urllib.request
from os.path import exists

if exists("./regular.csv"):
    !rm -rf regular.csv

f = urllib.request\
.urlretrieve ("https://docs.google.com/uc?export=download&id=1t3icaDgI5KeNEwNmaWFOYGYQtdY8NOMm",
              "regular.csv")

### ★★★ table 관련해서 Error 발생시 ★★★

아래의 코드를 이용해서, 등록 table을 지운 후 진행합시다.!

```
sqlContext.sql("SHOW tables").show()
sqlContext.dropTempTable("[지우고자 하는 table 이름]")
```

**task**

- 1. sqlContext.read.csv를 이용하여 ``regular.csv``를 DataFrame으로 만든 후 . ``registerTempTable("task1_table")``을 이용하여 table로 등록합니다. 그리고 SQL의 ``DESC``를 활용하여 구조를 확인합니다(5 point)

```
# task1 output
+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|  batter_id|   string|   null|
|batter_name|   string|   null|
|       year|   string|   null|
|       team|   string|   null|
|        avg|   string|   null|
|          G|   string|   null|
|         AB|   string|   null|
|          R|   string|   null|
|          H|   string|   null|
|         2B|   string|   null|
|         3B|   string|   null|
|         HR|   string|   null|
|         TB|   string|   null|
|        RBI|   string|   null|
|         SB|   string|   null|
|         CS|   string|   null|
|         BB|   string|   null|
|        HBP|   string|   null|
|         SO|   string|   null|
|        GDP|   string|   null|
+-----------+---------+-------+
only showing top 20 rows
```

In [10]:
sqlContext.read.csv("regular.csv", header = True).registerTempTable("task1_table")
#regular.csv를 읽어오고 읽은 자료를 task1_table이라는 이름으로 테이블을 만듦
#output


sqlContext.sql("DESC task1_table").show()#정렬하여 테이블을 보여줌

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|  batter_id|   string|   null|
|batter_name|   string|   null|
|       year|   string|   null|
|       team|   string|   null|
|        avg|   string|   null|
|          G|   string|   null|
|         AB|   string|   null|
|          R|   string|   null|
|          H|   string|   null|
|         2B|   string|   null|
|         3B|   string|   null|
|         HR|   string|   null|
|         TB|   string|   null|
|        RBI|   string|   null|
|         SB|   string|   null|
|         CS|   string|   null|
|         BB|   string|   null|
|        HBP|   string|   null|
|         SO|   string|   null|
|        GDP|   string|   null|
+-----------+---------+-------+
only showing top 20 rows



**task**
* 2. task1에서 생성된 ``task1_table``에서, 타율(avg) column의 type을 String에서 Double로 변환 후 ``registerTempTable("task2_table")``을 사용하여 새로운 table로 등록합니다(10 point)

```
# SQL에서 column type 변경하기
SELECT *, cast([column 이름] as [바꿀 type]) AS [새로운 column 이름] FROM [table]
# 여기서 기존의 column은 제거되지 않고 type이 변경된 column이 추가됩니다.

# task2 output
Row(col_name='avg_double', data_type='double', comment=None)
```

In [43]:
# 3-2 답 작성
a="""SELECT *,cast(avg as Double) AS avg_double FROM task1_table"""#a변수에 모든 행을 뽑고 avg를 double형태로 바꿔주고 이름을 avg_double로 바꾼다 table은 task1_table에서 가져온다
sqlContext.sql(a).registerTempTable("task2_table")
#a를 통해 새로운 테이블 task2_table을 만든다
# output
sqlContext.sql("DESC task2_table").collect()[-1]#정렬하고 마지막 것만 뽑아낸다

Row(col_name='avg_double', data_type='double', comment=None)

**task 3**
- 3. task2에서 생성된 ``task2_table``에서 타율(avg)이 0.300을 초과하는 모든 선수에 대해(중복포함), **각 타자의 3할을 친 횟수가 8 이상인 타자의 이름(batter_name)과 3할을 친 횟수(avgCount)를 구합니다**. 이에 덧붙여서, 3할을 친 횟수를 기준으로 내림차순을 적용하는 query를 작성하고 출력합니다.(10 point)

```
# task3 output
+-----------+--------+
|batter_name|avgCount|
+-----------+--------+
|     김태균|      13|
|     김주찬|      11|
|     이진영|      11|
|     이택근|      10|
|     손아섭|      10|
|     정근우|       9|
|     장성호|       9|
|     박용택|       9|
|     김동주|       9|
|     정성훈|       8|
|     이대호|       8|
|     최형우|       8|
|     박한이|       8|
|     김현수|       8|
+-----------+--------+
```

In [44]:
# 3-3 답 작성
query3 =  """SELECT batter_name, COUNT(avg_double) AS avgCount
FROM task2_table
WHERE avg_double > 0.300
GROUP BY batter_name
HAVING avgCount >= 8
ORDER BY avgCount DESC"""
#batter_name과 avg_double를 카운트하고 avgCount로 이름을 바꾸고 task2_table에서 가져오며 avg_double>0.300이상이여야하고 batter_name으로 묶으며
#avgCount>8이어야 하며 정렬을 한다.
# output
sqlContext.sql(query3).show()#sql조건 값을 적용하여 출력

+-----------+--------+
|batter_name|avgCount|
+-----------+--------+
|     김태균|      13|
|     이진영|      11|
|     김주찬|      11|
|     이택근|      10|
|     손아섭|      10|
|     김동주|       9|
|     장성호|       9|
|     정근우|       9|
|     박용택|       9|
|     정성훈|       8|
|     이대호|       8|
|     최형우|       8|
|     김현수|       8|
|     박한이|       8|
+-----------+--------+



## Exercise 4 

### 데이터를 찾고 DataFrame과 SQL을 적용해보자 (25 point)
- - -

지금까지 계속 똑같은 데이터(야구와 야구 그리고 baseball 등)를 사용하여 과제를 진행하였다. 

지겹지 않은가??? *(??? : 사실 내가 지겹다.)*

<br>

HW6 Exercise 4는 야구 데이터의 매너리즘에 빠진 당신을 위해 준비하였다. ㅋ

- - -

**task**
* 1. 공공데이터포털([https://www.data.go.kr/](https://www.data.go.kr/)), 데이콘([https://dacon.io/](https://dacon.io/)), kaggle([https://www.kaggle.com/](https://www.kaggle.com/)]) 등 각종 데이터를 제공하는 사이트에 접속하여 데이터를 찾습니다. 데이터의 형식은 상관 X. ``하지만, 우리가 학습해본 csv, json, parquet가 편하지 않을까...`` **데이터를 찾은 후 `sqlContext의 read.[형식에 맞게]`를 적용하여 DataFrame으로 저장한다**. 그 후, ``registerTempTable``을 사용하여 ``table``로 등록후, SQL을 사용하여 데이터의 구조를 출력합니다. (10 point)

    1. **[DataFrameReader 참고](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader)**

<br>

 
* 2. task1에서 불러온 데이터에 대해 설명하시오. (5 point)

    **<반드시 포함되어야할 내용>**
    
    1. 데이터의 출처, 데이터의 description, 데이터 활용 방안 등
    
<br>

* 3. task1의 데이터를 활용하여 자유롭게 결과를 출력하고, 왜 그러한 결과를 출력했는지와 코드에 대한 설명을 추가하세요.(10 point)

    **<반드시 포함되어야할 사항>**
    
    1. pyspark 함수가 아닌 **SQL 명령어 사용**. 또한, ``GROUP BY``, ``HAVING`` 반드시 포함되어야 함
    2. 결과에 대한 설명(print로 작성하시오)
    3. 주석 반드시 포함(코드에 대한 주석 #)
    
- - -
**출력 예시**

![png](http://www.artinsight.co.kr/data/news/1811/1988088618_t0JncQ9w_13.jpg)

**task**
* 1. 공공데이터포털([https://www.data.go.kr/](https://www.data.go.kr/)), 데이콘([https://dacon.io/](https://dacon.io/)), kaggle([https://www.kaggle.com/](https://www.kaggle.com/)]) 등 각종 데이터를 제공하는 사이트에 접속하여 데이터를 찾습니다. 데이터의 형식은 상관 X. ``하지만, 우리가 학습해본 csv, json, parquet가 편하지 않을까...`` **데이터를 찾은 후 `sqlContext의 read.[형식에 맞게]`를 적용하여 DataFrame으로 저장한다**. 그 후, ``registerTempTable``을 사용하여 ``table``로 등록후, SQL을 사용하여 데이터의 구조를 출력합니다. (10 point)

    1. **[DataFrameReader 참고](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader)**


```
sqlContext.read.csv("./전국금연구역표준데이터.csv", header = True, encoding="EUC-kR").registerTempTable("non_smoking")

★ task1 example output ★
★ tempTable SHOW★
+--------+-----------+-----------+
|database|  tableName|isTemporary|
+--------+-----------+-----------+
|        |non_smoking|       true|
|        |     people|       true|
|        |task1_table|       true|
|        |task2_table|       true|
|        |    weather|       true|
+--------+-----------+-----------+

★ DESC★
+----------------+---------+-------+
|        col_name|data_type|comment|
+----------------+---------+-------+
|      금연구역명|   string|   null|
|금연구역범위상세|   string|   null|
|          시도명|   string|   null|
|        시군구명|   string|   null|
|    금연구역구분|   string|   null|
+----------------+---------+-------+
only showing top 5 rows
```

In [173]:
# 4-1 답안 작성
# example code
#sqlContext.read.csv("./전국금연구역표준데이터.csv", header = True, encoding="EUC-kR").registerTempTable("non_smoking")
sqlContext.read.csv("./accroad1.csv", header = True, encoding="EUC-kR").registerTempTable("acc_road1")
#내가 다운받은 accroad1의 csv를 읽고 acc_road1이라는 이름으로 테이블을 만듦



# example output
# table명은 자유롭게 
print("★ tempTable SHOW★")#tempTalbeshow를 출력
sqlContext.sql("SHOW tables").show()#SHOWTables을 통해 내가 가지고 있는 csv들을 출력
print("★ DESC★")#정렬
sqlContext.sql("DESC acc_road1").show(5)#내가 다운받은 acc_road1을 정렬하여 5개만 보여준다

★ tempTable SHOW★
+--------+-----------+-----------+
|database|  tableName|isTemporary|
+--------+-----------+-----------+
|        |  acc_road1|       true|
|        |    geonmoo|       true|
|        |  one_table|       true|
|        | one_table1|       true|
|        |  one_talbe|       true|
|        |task2_table|       true|
+--------+-----------+-----------+

★ DESC★
+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
|        name|   string|   null|
|        year|   string|   null|
|    location|   string|   null|
|locationcode|   string|   null|
|locationname|   string|   null|
+------------+---------+-------+
only showing top 5 rows



**task**
* 2. task1에서 불러온 데이터에 대해 설명하시오. (5 point)

    **<반드시 포함되어야할 내용>**
    
    1. 데이터의 출처, 데이터의 description, 데이터 활용 방안 등

In [170]:
# 4-2 답안 작성
print("[4-2 답안 작성] \n")
print("데이터의 출처 : https://www.data.go.kr/dataset/15003493/fileData.do")
print("데이터의 description : 도로교통공단에서 교통사고 다발지역에 따른 부상자수")
print("데이터 활용 방안 :이 데이터를 통해 사고다발 지역 구간에서 운전을 조심하도록 경각시킬 수 있음 ")
print("기타 등등 : 그 외에도 위험한 지역을 정부가 파악하여 위험지역에 사고가 나지 않도록 인력배치 및 기구들을 설치할 수 있는 좋은 통계자료")

[4-2 답안 작성] 

데이터의 출처 : https://www.data.go.kr/dataset/15003493/fileData.do
데이터의 description : 도로교통공단에서 교통사고 다발지역에 따른 부상자수
데이터 활용 방안 :이 데이터를 통해 사고다발 지역 구간에서 운전을 조심하도록 경각시킬 수 있음 
기타 등등 : 그 외에도 위험한 지역을 정부가 파악하여 위험지역에 사고가 나지 않도록 인력배치 및 기구들을 설치할 수 있는 좋은 통계자료


**task**
* 3. task1의 데이터를 활용하여 자유롭게 결과를 출력하고, 왜 그러한 결과를 출력했는지와 코드에 대한 설명을 추가하세요.(10 point)

    **<반드시 포함되어야할 사항>**
    
    1. pyspark 함수가 아닌 **SQL 명령어 사용**. 또한, ``GROUP BY``, ``HAVING`` 반드시 포함되어야 함
    2. 결과에 대한 설명(print로 작성하시오)
    3. 주석 반드시 포함(코드에 대한 주석 #)

# 수고ㅋ!

In [215]:

query1="""SELECT name, avg(avg1) as avg_dead
FROM acc_road1
GROUP BY name 
HAVING avg(avg1) >= 1
""" #acc_road1 테이블 사용 name,avg1의 평균을 avg_dead로 바꿔서 뽑아냄
    #name별로 묶어주고 avg1의 평균이 1이상 큰 것을 출력
sqlContext.sql(query1).show(5)#수가 많아지므로 5개만 출력해서 보여줌
print("사고지역관리별 평균 사상자 수의 출력")
# output

+-------+------------------+
|   name|          avg_dead|
+-------+------------------+
|2019034|2.0714285714285716|
|2016147| 5.668341708542713|
|2014105| 4.021739130434782|
|2014110|3.4430379746835444|
|2016040| 2.046511627906977|
+-------+------------------+
only showing top 5 rows

사고지역관리별 평균 사상자 수의 출력
